In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold

from scipy.signal import find_peaks
from scipy.stats import mode
import numpy as np
import pickle
import matplotlib.pyplot as plt

2024-01-10 08:33:20.157228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
# Load input/label to models
# RESPIRATION
with open('resp_input.pkl', 'rb') as file:
    x = pickle.load(file)
    
with open('resp_label.pkl', 'rb') as file:
    y = pickle.load(file)

# CARDIAC
# with open('cardiac_input.pkl', 'rb') as file:
#     x = pickle.load(file)

# with open('cardiac_label.pkl', 'rb') as file:
#     y = pickle.load(file)

In [4]:
nanmask = np.isnan(y)
print(f'#Nan Labels: {np.sum(nanmask)}\n#Remaining Samples: {np.sum(~nanmask)}')
y = y[~nanmask]
x = x[~nanmask]

#Nan Labels: 590
#Remaining Samples: 3304


In [5]:
### GOAL is to achieve 95% classification accuracy on 10-fold CV

# Step 1: Experiment with different param. + hyperparam.: layers, kernel, stride, filter size ( see https://keras.io/api/layers/convolution_layers/convolution1d/ )
# Step 2: Experiment with different architectures

test_loss, test_acc = [], []

kernel = 5
stride = 3
kfold = KFold(n_splits=10) #, shuffle=True)
i = 0
for train, test in kfold.split(x, y):
    # Data normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(x[train])
    X_test = scaler.transform(x[test])

    y_train = y[train]
    y_test = y[test]
    
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


    # TO DO: Experiment with different model architectures
    model = Sequential([
        layers.Conv1D(64, kernel, stride, activation='relu', input_shape=X_train.shape[1:]),
        layers.Conv1D(32, kernel, stride, activation='relu'),
        layers.Conv1D(16, kernel, stride, activation='relu'),
        layers.Conv1D(8, kernel, stride, activation='relu'),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
        
    # Define the early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Metric to monitor (e.g., validation loss)
        patience=5,           # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore model weights to the best achieved during training
        verbose=1  # Set to 1 to see messages about the early stopping process
    )
    
    optimizer = optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    history = model.fit(
        X_train,
        y_train,
        epochs=25,      
        validation_split=0.2,
        batch_size=2,
        callbacks=[early_stopping]  # Add the early stopping callback
    )
    
    loss, accuracy = model.evaluate(X_test, y_test, batch_size=1)
    print(f"Fold {i} Test Accuracy: {accuracy * 100:.2f}%")
    i = i + 1
    test_loss.append(loss)
    test_acc.append(accuracy)

print("{} Fold Accuracy: {:.2f}%\u00b1{:.2f}".format(i, np.mean(test_acc)*100, np.std(test_acc)*100))

Epoch 1/25
1189/1189 [==============================] - 17s 14ms/step - loss: 0.5120 - accuracy: 0.7733 - val_loss: 0.4805 - val_accuracy: 0.7664
Epoch 2/25
1189/1189 [==============================] - 16s 14ms/step - loss: 0.3850 - accuracy: 0.8272 - val_loss: 0.4558 - val_accuracy: 0.7731
Epoch 3/25
1189/1189 [==============================] - 16s 14ms/step - loss: 0.3227 - accuracy: 0.8654 - val_loss: 0.4526 - val_accuracy: 0.7849
Epoch 4/25
1189/1189 [==============================] - 16s 14ms/step - loss: 0.2478 - accuracy: 0.8957 - val_loss: 0.5329 - val_accuracy: 0.7580
Epoch 5/25
1189/1189 [==============================] - 16s 14ms/step - loss: 0.1902 - accuracy: 0.9188 - val_loss: 0.5782 - val_accuracy: 0.7647
Epoch 6/25
1189/1189 [==============================] - 16s 14ms/step - loss: 0.1618 - accuracy: 0.9441 - val_loss: 2.8049 - val_accuracy: 0.6958
Epoch 7/25
1189/1189 [==============================] - 16s 14ms/step - loss: 0.1211 - accuracy: 0.9613 - val_loss: 0.8458 -

In [6]:
test_acc

[0.7432023882865906,
 0.7764350175857544,
 0.9063444137573242,
 0.791540801525116,
 0.7696969509124756,
 0.8515151739120483,
 0.8696969747543335,
 0.8363636136054993,
 0.8454545736312866,
 0.7787878513336182]